In [1]:
import torch
import sys 

sys.path.append('/src/detector/yolov5')
from models.common import DetectMultiBackend
from utils.torch_utils import select_device


In [2]:
device  = select_device('0')
device

YOLOv5 🚀 2022-10-19 Python-3.7.13 torch-1.12.1+cu102 CUDA:0 (Quadro RTX 5000, 16125MiB)



device(type='cuda', index=0)

In [3]:
weights = "/PhamMinhDuc_STEAM/src/yolov5/PhamMinhDuc_STEAM/yolov5x_vehicle/weights/last.pt"
data =  "/PhamMinhDuc_STEAM/src/yolov5/vehicle.yaml"

In [4]:
model = DetectMultiBackend(weights, device=device, data=data)
model.eval()

Fusing layers... 
Model summary: 322 layers, 86193601 parameters, 0 gradients, 203.8 GFLOPs


DetectMultiBackend(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 80, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv3): Conv(
          (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): Sequential(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(80, 80, kernel_size=(1, 1), stride=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): 

In [5]:
source = "/PhamMinhDuc_STEAM/DATA/video_test/IMG_4393.MOV"
imgz_size = (640, 640)
stride, names, pt = model.stride, model.names, model.pt
vid_stride=1

In [6]:
from utils.dataloaders import LoadImages
from utils.general import non_max_suppression, scale_boxes

In [7]:
data = LoadImages(source, img_size=imgz_size, stride=stride, auto=pt, vid_stride=vid_stride)

In [8]:
data

In [9]:
import matplotlib.pyplot as plt
import cv2
from IPython.display import Image

In [10]:
for path, im, im0s, vid_cap, s in data:
    img = torch.from_numpy(im).to(device).float()
    img /= 255.0
    if img.ndimension() == 3:
        img = img[None]
    pred = model(img, augment=False)
    break

In [11]:
pred = non_max_suppression(pred, 0.8, 0.3, classes=None, agnostic=False)

In [12]:

for i, det in enumerate(pred):
        if len(det):
            p, im0, frame = path, im0s.copy(), getattr(data, 'frame', 0)
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]] 
            det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], im0.shape).round()
            print(det)
            
            
            for *xyxy, conf, cls in reversed(det):
                # xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()
                label = f'{names[int(cls)]} {conf:.2f}'
                # class_id = int(cls)
                # # convert to coordinates in original image
                x1 = int(xyxy[0].item())
                y1 = int(xyxy[1].item())
                x2 = int(xyxy[2].item())
                y2 = int(xyxy[3].item())
                # # conf to string
                conf = str(conf.item())
                
                
                # plot orginal image
                im0 = cv2.rectangle(im0, (x1, y1), (x2, y2), (255, 0, 0), 3)
                
                # save image ploted
            cv2.imwrite("/PhamMinhDuc_STEAM/DATA/video_test/IMG_4393.jpg", im0)
            break

tensor([[3.00000e+00, 6.62000e+02, 5.55000e+02, 1.07700e+03, 9.66427e-01, 2.00000e+00],
        [8.69000e+02, 9.28000e+02, 9.58000e+02, 1.07800e+03, 9.63877e-01, 0.00000e+00],
        [1.15000e+03, 7.42000e+02, 1.35000e+03, 9.10000e+02, 9.63497e-01, 1.00000e+00],
        [1.11800e+03, 8.35000e+02, 1.19500e+03, 1.02100e+03, 9.63045e-01, 0.00000e+00],
        [1.70900e+03, 7.18000e+02, 1.79600e+03, 8.50000e+02, 9.60978e-01, 0.00000e+00],
        [1.18300e+03, 3.77000e+02, 1.36500e+03, 5.18000e+02, 9.59045e-01, 2.00000e+00],
        [8.35000e+02, 6.97000e+02, 9.03000e+02, 8.41000e+02, 9.58802e-01, 0.00000e+00],
        [9.49000e+02, 7.62000e+02, 1.02800e+03, 9.20000e+02, 9.57162e-01, 0.00000e+00],
        [1.01700e+03, 6.02000e+02, 1.05900e+03, 7.08000e+02, 9.54943e-01, 0.00000e+00],
        [7.02000e+02, 4.89000e+02, 8.24000e+02, 6.42000e+02, 9.45439e-01, 3.00000e+00],
        [4.10000e+01, 4.73000e+02, 8.80000e+01, 5.64000e+02, 9.44093e-01, 0.00000e+00],
        [1.01300e+03, 5.08000e+0